In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/liste_mandat.csv") 
#on récupère le csv de la liste des mandats

#### Analyse des NAN

Les nan de département sont très majoitairement issu de la révolution :

In [93]:
df[df['Département'].isna()].count()

Unnamed: 0          1330
Département            0
Groupe              1302
Législature         1317
Mandat              1317
Régime politique    1314
nom du député       1330
dtype: int64

In [95]:
df[df['Département'].isna()][df['Régime politique']!='Révolution'].count()

<ipython-input-95-54b5895b7c85>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['Département'].isna()][df['Régime politique']!='Révolution'].count()


Unnamed: 0          30
Département          0
Groupe               6
Législature         17
Mandat              17
Régime politique    14
nom du député       30
dtype: int64

Pour les régime politique, on a les cas super-pathologique + des personnes de cette législature un peu perdu

In [98]:
df[df['Régime politique'].isna()]

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,nom du député
1050,0,NaN,2,(Législature en cours),Depuis le 26 octobre 2020 (mandat en cours),NaN,François Baroin
24363,0,NaN,NaN,NaN,NaN,NaN,"Charles, André, Marie, Joseph de Gaulle"
24379,0,NaN,NaN,NaN,NaN,NaN,"Jean, Joseph, Paul, Augustin Dessolle"
24412,0,NaN,NaN,(Législature en cours),Depuis le 26 octobre 2020 (mandat en cours),NaN,"Joseph, François, Marie Kermorial"
28099,0,NaN,NaN,(Législature en cours),Depuis le 26 octobre 2020 (mandat en cours),NaN,"Emmanuel, Joseph Sieyès"
32530,0,NaN,NaN,NaN,NaN,NaN,Alexandre Bachelet
32531,0,NaN,NaN,NaN,NaN,NaN,Henry Senes
32532,0,NaN,NaN,NaN,NaN,NaN,Camille Rolland
32533,0,NaN,NaN,NaN,NaN,NaN,Jean-Pierre Rambaut
32534,0,NaN,NaN,NaN,NaN,NaN,Georges Pezières


La ce n'est que les cas super pathologiques 

In [99]:
df[df['Mandat'].isna()]

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,nom du député
24363,0,NaN,NaN,NaN,NaN,NaN,"Charles, André, Marie, Joseph de Gaulle"
24379,0,NaN,NaN,NaN,NaN,NaN,"Jean, Joseph, Paul, Augustin Dessolle"
32530,0,NaN,NaN,NaN,NaN,NaN,Alexandre Bachelet
32531,0,NaN,NaN,NaN,NaN,NaN,Henry Senes
32532,0,NaN,NaN,NaN,NaN,NaN,Camille Rolland
32533,0,NaN,NaN,NaN,NaN,NaN,Jean-Pierre Rambaut
32534,0,NaN,NaN,NaN,NaN,NaN,Georges Pezières
32535,0,NaN,NaN,NaN,NaN,NaN,Victor Le Gorgeu
32536,0,NaN,NaN,NaN,NaN,NaN,François Labrousse
32537,0,NaN,NaN,NaN,NaN,NaN,Paul Fleurot


On a beaucoup de nan dans les groupes mais c'est plus normal.

In [103]:
df[df['Groupe'].isna()].count()

Unnamed: 0          4154
Département         4126
Groupe                 0
Législature         4141
Mandat              4141
Régime politique    4139
nom du député       4154
dtype: int64